In [1]:
import os
import glob
import pickle
import datetime
import numpy as np
from astropy.io import fits
from photutils import DAOStarFinder, CircularAperture
import photutils.psf
import matplotlib.pyplot as plt
from astropy.visualization import LogStretch, AsinhStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from scipy.signal import peak_widths
import astropy.units as u
import pandas as pd
from scipy import stats
from calibFuncs import *

In [2]:
datadir = 'C:\\Users\\willi\\Dropbox (Amherst College)\\Research\\Follette-Lab\\Thesis-Data\William\\trapezium_data\\visao_astro'
uniques, VisAO_science, VisAO_darks = sortData(datadir, instrument='VisAO', filesdeep='*\\*\\', returntab=True)

  0%|                                                                              | 16/33363 [00:00<03:32, 156.93it/s]

sorting individual images


100%|█████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 386.74it/s]

sorting unique datasets into lists


In [3]:
# data takes a long time to sort so pickle result
filename = 'visao_science.pkl'
outfile = open(filename,'wb')
pickle.dump(VisAO_science,outfile)
outfile.close()

filename = 'visao_uniques.pkl'
outfile = open(filename,'wb')
pickle.dump(uniques,outfile)
outfile.close()

filename = 'visao_darks.pkl'
outfile = open(filename,'wb')
pickle.dump(VisAO_darks,outfile)
outfile.close()

In [4]:
# read in pickle
infile = open('visao_science.pkl','rb')
VisAO_science = pickle.load(infile)
infile.close()

infile = open('visao_uniques.pkl','rb')
uniques = pickle.load(infile)
infile.close()

infile = open('visao_darks.pkl','rb')
VisAO_darks = pickle.load(infile)
infile.close()

In [51]:
uniques

['2014-11-10&SDI-H-alpha&Theta1_Ori_C',
 '2014-11-15&zprime&Theta1_Ori_C',
 '2014-11-16&zprime&Theta1_Ori_C',
 '2014-11-17&SDI-H-alpha&Unknown',
 '2014-11-29&SDI-H-alpha&Theta1_Ori_C',
 '2015-05-15&zprime&HD_165054',
 '2015-12-11&SDI-H-alpha&Theta1_Ori_B',
 '2016-11-17&iprime&Theta1_Ori_C',
 '2016-11-17&zprime&Theta1_Ori_C',
 '2016-11-17&SDI-H-alpha&Theta1_Ori_C',
 '2018-04-27&SDI-H-alpha&Baades_Window',
 '2018-04-27&unknown&Baades_Window',
 '2018-04-27&SDI-H-alpha&70_Oph',
 '2018-04-26&SDI-H-alpha&Theta1_Ori_B',
 '2018-04-26&unknown&Theta1_Ori_B',
 '2018-04-26&rprime&Theta1_Ori_B',
 '2018-04-26&zprime&Theta1_Ori_B']

In [55]:
for i in range(len(VisAO_science)):
    dataset = VisAO_science[i]
    try:
        first_im = dataset[0]
    except IndexError:
        break
    first_im_root = first_im.replace(first_im.split('\\')[-1], 'science\\')
    if os.path.exists(first_im_root):
        pass
    else:
        os.mkdir(first_im_root)
    for sci_im in dataset:
        imname = sci_im.split('\\')[-1]
        newpath = sci_im.replace(imname, 'science\\'+imname)
        os.rename(sci_im, newpath)

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'C:\\Users\\willi\\Dropbox (Amherst College)\\Research\\Follette-Lab\\Thesis-Data\\William\\trapezium_data\\visao_astro\\visao_20141109_10\\tet1OriC_Ha_01\\V47_20141110091732005115.fits.gz' -> 'C:\\Users\\willi\\Dropbox (Amherst College)\\Research\\Follette-Lab\\Thesis-Data\\William\\trapezium_data\\visao_astro\\visao_20141109_10\\tet1OriC_Ha_01\\science\\V47_20141110091732005115.fits.gz'